In [7]:
library(tidyverse)
library(lme4)
library(lmerTest)
library(ggpubr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ forcats 0.5.0
✔ readr   1.4.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::add_rownames() masks huxtable::add_rownames()
✖ dplyr::filter()       masks rstatix::filter(), stats::filter()
✖ dplyr::lag()          masks stats::lag()
✖ ggplot2::theme_grey() masks huxtable::theme_grey()

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Registered S3 methods overwritten by 'lme4':
  method                          from
  cooks.distance.influence.merMod car 
  influence.merMod                car 
  dfbeta.influence.merMod         car 
  dfbetas.influence.merMod        car 


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The 

In [2]:
tables_path = "/home/marc/ba-thesis/input/tables"

mean_amplitudes_df = readr::read_csv("../data/mean_amplitudes.csv")
mean_amplitudes_df = mean_amplitudes_df %>% mutate(Participant = factor(Participant), SOA = factor(SOA), Electrode = factor(Electrode), Condition = factor(Condition), StimulusType = factor(StimulusType))

n1_mean_amplitudes_df = readr::read_csv("../data/n1_mean_amplitudes.csv")
n1_mean_amplitudes_df = n1_mean_amplitudes_df %>% mutate(Participant = factor(Participant), SOA = factor(SOA), Electrode = factor(Electrode), Condition = factor(Condition), StimulusType = factor(StimulusType))
n1_mean_amplitudes_df


── Column specification ────────────────────────────────────────────────────────
cols(
  SOA = col_double(),
  Participant = col_character(),
  Condition = col_character(),
  StimulusType = col_character(),
  MeanAmplitude = col_double(),
  Electrode = col_character()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  SOA = col_double(),
  Participant = col_character(),
  Condition = col_character(),
  StimulusType = col_character(),
  MeanAmplitude = col_double(),
  Electrode = col_character()
)




SOA,Participant,Condition,StimulusType,MeanAmplitude,Electrode
<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>
100,100_0,random,A-1,-1.530263e-07,FZ
100,100_1,random,A-1,-4.071953e-07,FZ
100,100_2,random,A-1,-1.431155e-06,FZ
100,100_3,random,A-1,1.515026e-07,FZ
100,100_4,random,A-1,-6.722830e-07,FZ
100,100_5,random,A-1,2.200396e-07,FZ
100,100_6,random,A-1,6.661060e-07,FZ
100,100_7,random,A-1,-3.136040e-06,FZ
100,100_8,random,A-1,-1.212225e-06,FZ


In [3]:


# mean_amplitudes_150 = mean_amplitudes_df %>% filter(SOA == "150" & Electrode == "fronto_pooled") %>% droplevels()

# model1 = lmer(MeanAmplitude ~ Condition * StimulusType + (1|Participant) + (1|Participant:Condition) + (1|Participant:StimulusType), data = mean_amplitudes_150)
# model2 = aov(MeanAmplitude~Condition*StimulusType+Error(Participant/(Condition*StimulusType)), data=mean_amplitudes_150)
# model3 = ez::ezANOVA(mean_amplitudes_150, dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType), return_aov=T)
# model4 = nlme::lme(MeanAmplitude ~ Condition * StimulusType, random=~1|Participant, correlation=nlme::corCompSymm(form=~1|Participant), data=mean_amplitudes_150)
# model5 = lmer(MeanAmplitude ~ (1|Participant) + (1|Participant:Condition) + (1|Participant:StimulusType), data = mean_amplitudes_150)

# car::Anova(model1) # car on lmer
# anova(model1) # lmerTest
# summary(model2) # aov (always type I)
# model3$ANOVA # ez
# anova(model4)


# #ggplot(data=mean_amplitudes_100) +
# #    geom_boxplot(aes(x = Condition, y = MeanAmplitude, color = StimulusType))

In [47]:
# 
desc_table = mean_amplitudes_df %>% filter(Electrode == "fronto_pooled") %>% droplevels() %>%
    group_by(SOA, Condition, StimulusType) %>%
    summarise(Mean = mean(MeanAmplitude)*10e5, SD = sd(MeanAmplitude)*10e5) %>%
    hux() %>%
    merge_repeated_rows(col=c(1,2)) %>%
    theme_article()

desc_table_mas = mean_amplitudes_df %>% filter(Electrode == "mastoids_pooled") %>% droplevels() %>%
    group_by(SOA, Condition, StimulusType) %>%
    summarise(Mean = mean(MeanAmplitude)*10e5, SD = sd(MeanAmplitude)*10e5) %>% ungroup() %>%
    select(Mean, SD) %>%
    hux() %>%
    merge_repeated_rows(col=c(1,2)) %>%
    theme_article()

desc_table_final = desc_table %>%
    cbind()

write(to_latex(cbind(desc_table,desc_table_mas)), paste(tables_path, "desc_table.tex", sep="/"))


print_md(cbind(desc_table,desc_table_mas))



`summarise()` regrouping output by 'SOA', 'Condition' (override with `.groups` argument)

`summarise()` regrouping output by 'SOA', 'Condition' (override with `.groups` argument)

Warning message in to_md.huxtable(ht, ...):
“Markdown cannot handle cells with colspan/rowspan > 1”
---------------------------------------------------------------
 **SOA** **Condit** **Stimul** **Mean** **SD** **Mean** **SD** 
         **ion**    **usType**                                 
-------- ---------- ---------- -------- ------ -------- -------
 100     predictabl A            -0.431   1.23   -0.052   1.51 
         e                                                     
                    B           -0.0477   1.22   -0.184   1.56 
                                                               
         random     A            -0.225   1.82    -1.04   2.64 
                                                               
                    B            -0.583   2.16   -0.296   3.23 
                

In [48]:
library(rstatix)
library(xtable)
library(huxtable)
library(stringr)

fm_pvalue = function(p) {
    out = ifelse(p < .001, "\u003C.001", sprintf("%.3f",p))
    out = stringr::str_replace(out, fixed("0."), ".")
    
    return(out)
}

variables = list()

### 100 ms x 150ms
## ANOVA with pooled electrodes
anova_00_data = mean_amplitudes_df %>% filter(Electrode == "fronto_pooled") %>% droplevels() 
anova_00_table =    anova_00_data %>% 
                        anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType), between = c(SOA)) %>%
                        get_anova_table(correction = "auto")
anova_00_table$Effect = str_replace_all(anova_00_table$Effect, ":", " x ")
variables$anova_00_soa = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$, $\\eta_{gen}=%.4f$", anova_00_table$DFn[[1]], anova_00_table$DFd[[1]], anova_00_table$F[[1]], anova_00_table$p[[1]], anova_00_table$ges[[1]])
print(xtable(anova_00_table), label = "anova_00_table", include.rownames = F, tabular.environment = "tabulary", width = "\\textwidth", file = paste(tables_path, "anova_00.tex", sep="/"))

### 100 ms
## ANOVA for 100 ms with pooled electrodes (fronto x mastoids)
anova_02_100_data = mean_amplitudes_df %>% filter(SOA == "100") %>% filter(Electrode == "fronto_pooled" | Electrode == "mastoids_pooled") %>% droplevels() 

anova_02_100_table =    anova_02_100_data %>% 
                        anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType, Electrode)) %>%
                        get_anova_table(correction = "auto")
anova_02_100_table$Effect = str_replace_all(anova_02_100_table$Effect, ":", " x ")
variables$anova_02_100_condition = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[1]], anova_02_100_table$DFd[[1]], anova_02_100_table$F[[1]], anova_02_100_table$p[[1]])
variables$anova_02_100_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[2]], anova_02_100_table$DFd[[2]], anova_02_100_table$F[[2]], anova_02_100_table$p[[2]])
variables$anova_02_100_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[3]], anova_02_100_table$DFd[[3]], anova_02_100_table$F[[3]], anova_02_100_table$p[[3]])
variables$anova_02_100_condition_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[4]], anova_02_100_table$DFd[[4]], anova_02_100_table$F[[4]], anova_02_100_table$p[[4]])
variables$anova_02_100_condition_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[5]], anova_02_100_table$DFd[[5]], anova_02_100_table$F[[5]], anova_02_100_table$p[[5]])
variables$anova_02_100_stimulustype_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[6]], anova_02_100_table$DFd[[6]], anova_02_100_table$F[[6]], anova_02_100_table$p[[6]])
variables$anova_02_100_condition_stimulustype_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[7]], anova_02_100_table$DFd[[7]], anova_02_100_table$F[[7]], anova_02_100_table$p[[7]])
print(xtable(anova_02_100_table), include.rownames = F, tabular.environment = "tabulary", width = "\\textwidth", file = paste(tables_path, "anova_01_100.tex", sep="/"))


### 100 ms
## ANOVA for 100 ms with pooled electrodes
anova_01_100_data = mean_amplitudes_df %>% filter(SOA == "100" & Electrode == "fronto_pooled") %>% droplevels() 
anova_01_100_table =    anova_01_100_data %>% 
                        anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType)) %>%
                        get_anova_table()
anova_01_100_table$Effect = str_replace_all(anova_01_100_table$Effect, ":", " x ")
variables$anova_01_100_condition = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_100_table$DFn[[1]], anova_01_100_table$DFd[[1]], anova_01_100_table$F[[1]], anova_01_100_table$p[[1]])
variables$anova_01_100_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_100_table$DFn[[2]], anova_01_100_table$DFd[[2]], anova_01_100_table$F[[2]], anova_01_100_table$p[[2]])
variables$anova_01_100_condition_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_100_table$DFn[[3]], anova_01_100_table$DFd[[3]], anova_01_100_table$F[[3]], anova_01_100_table$p[[3]])
caption  = "Results of the 3-way ANOVA (condition x stimulus x electrode) for repeated measures conducted on the mean ERP-amplitudes (time window 111 - 161 ms) at electrode Fz (upper section). The significant interaction between the three factors included was further analyzed by 2-way ANOVAS (stimulus x electrode) conducted separately for the random condition (middle section) and the predictable condition (lower section)."
print(xtable(anova_01_100_table, correction = "auto", caption = caption), include.rownames = F, tabular.environment = "tabulary", width = "\\textwidth", file = paste(tables_path, "anova_02_100.tex", sep="/"))

## ANOVA for 100 ms for fronto
anova_03_100_data = mean_amplitudes_df %>% filter(SOA == "100" & Electrode == "fronto_pooled") %>% droplevels() 
anova_03_100_table = anova_03_100_data %>% 
                     anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType)) %>%
                     get_anova_table(correction = "auto")
anova_03_100_table$Effect = str_replace_all(anova_03_100_table$Effect, ":", " x ")
caption  = "Results from two-way ANOVA for 100 ms (only fronto)"
print(xtable(anova_03_100_table, caption=caption), include.rownames = F, tabular.environment = "tabulary", booktabs = TRUE, width = "\\textwidth", file = paste(tables_path, "anova_03_100.tex", sep="/"))

## ANOVA for 100 ms for mastioid
anova_04_100_data =  mean_amplitudes_df %>% filter(SOA == "100" & Electrode == "mastoids_pooled") %>% droplevels() 
anova_04_100_table = anova_04_100_data %>% 
                     anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType)) %>%
                     get_anova_table(correction = "auto")
anova_04_100_table$Effect = str_replace_all(anova_04_100_table$Effect, ":", " x ")
caption  = "Results from two-way ANOVA for 100 ms with non-pooled electrodes"
print(xtable(anova_04_100_table, caption=caption), include.rownames = F, tabular.environment = "tabulary", booktabs = TRUE, width = "\\textwidth", file = paste(tables_path, "anova_04_100.tex", sep="/"))

### 150 ms
## ANOVA for 150 ms with pooled electrodes (fronto x mastoids)
anova_02_150_data = mean_amplitudes_df %>% filter(SOA == "150" & (Electrode == "fronto_pooled" | Electrode == "mastoids_pooled")) %>% droplevels() 

anova_02_150_table =    anova_02_150_data %>% 
                        anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType, Electrode)) %>%
                        get_anova_table(correction = "auto")
anova_02_150_table$Effect = str_replace_all(anova_02_150_table$Effect, ":", " x ")
variables$anova_02_150_condition = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[1]], anova_02_150_table$DFd[[1]], anova_02_150_table$F[[1]], anova_02_150_table$p[[1]])
variables$anova_02_150_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[2]], anova_02_150_table$DFd[[2]], anova_02_150_table$F[[2]], anova_02_150_table$p[[2]])
variables$anova_02_150_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[3]], anova_02_150_table$DFd[[3]], anova_02_150_table$F[[3]], anova_02_150_table$p[[3]])
variables$anova_02_150_condition_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[4]], anova_02_150_table$DFd[[4]], anova_02_150_table$F[[4]], anova_02_150_table$p[[4]])
variables$anova_02_150_condition_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[5]], anova_02_150_table$DFd[[5]], anova_02_150_table$F[[5]], anova_02_150_table$p[[5]])
variables$anova_02_150_stimulustype_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[6]], anova_02_150_table$DFd[[6]], anova_02_150_table$F[[6]], anova_02_150_table$p[[6]])
variables$anova_02_150_condition_stimulustype_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[7]], anova_02_150_table$DFd[[7]], anova_02_150_table$F[[7]], anova_02_150_table$p[[7]])
print(xtable(anova_02_150_table), include.rownames = F, tabular.environment = "tabulary", width = "\\textwidth", file = paste(tables_path, "anova_01_150.tex", sep="/"))


## ANOVA for 150 ms with pooled electrodes
anova_01_150_data = mean_amplitudes_df %>% filter(SOA == "150" & Electrode == "fronto_pooled") %>% droplevels() 
anova_01_150_table =    anova_01_150_data %>% 
                        anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType)) %>%
                        get_anova_table()
anova_01_150_table$Effect = str_replace_all(anova_01_150_table$Effect, ":", " x ")
variables$anova_01_150_condition = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_150_table$DFn[[1]], anova_01_150_table$DFd[[1]], anova_01_150_table$F[[1]], anova_01_150_table$p[[1]])
variables$anova_01_150_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_150_table$DFn[[2]], anova_01_150_table$DFd[[2]], anova_01_150_table$F[[2]], anova_01_150_table$p[[2]])
variables$anova_01_150_condition_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_150_table$DFn[[3]], anova_01_150_table$DFd[[3]], anova_01_150_table$F[[3]], anova_01_150_table$p[[3]])
print(xtable(anova_01_150_table, correction = "auto"), include.rownames = F, tabular.environment = "tabulary", width = "\\textwidth", file = paste(tables_path, "anova_01_150.tex", sep="/"))



## ANOVA for 150 ms for fronto
anova_03_150_data = mean_amplitudes_df %>% filter(SOA == "150" & Electrode == "fronto_pooled") %>% droplevels() 
anova_03_150_table = anova_03_150_data %>% 
                     anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType)) %>%
                     get_anova_table(correction = "auto")
anova_03_150_table$Effect = str_replace_all(anova_03_150_table$Effect, ":", " x ")
caption  = "Results from two-way ANOVA for 150 ms (only fronto)."
print(xtable(anova_03_150_table, caption=caption), include.rownames = F, tabular.environment = "tabulary", booktabs = TRUE, width = "\\textwidth", file = paste(tables_path, "anova_03_150.tex", sep="/"))

## ANOVA for 150 ms for mastioid
anova_04_150_data =  mean_amplitudes_df %>% filter(SOA == "150" & Electrode == "mastoids_pooled") %>% droplevels() 
anova_04_150_table = anova_04_150_data %>% 
                     anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType)) %>%
                     get_anova_table(correction = "auto")
anova_04_150_table$Effect = str_replace_all(anova_04_150_table$Effect, ":", " x ")
caption  = "Results from two-way ANOVA for 150 ms (only mastioids)."
print(xtable(anova_04_150_table, caption=caption), include.rownames = F, tabular.environment = "tabulary", booktabs = TRUE, width = "\\textwidth", file = paste(tables_path, "anova_04_150.tex", sep="/"))


## Create more complex tables

anova_02_full_caption  = "Results of the 3-way ANOVA (condition x stimulus x electrode) for repeated measures conducted on the mean ERP-amplitudes (time window 111 - 161 ms) at electrode Fz (upper section). The significant interaction between the three factors included was further analyzed by 2-way ANOVAS (stimulus x electrode) conducted separately for the random condition (middle section) and the predictable condition (lower section)."

anova_02_full_table = rbind(anova_02_100_table %>% add_column(SOA = paste("100", "ms"), .before = 1), anova_02_150_table %>% add_column(SOA = paste("150", "ms"), .before =1)) %>%
                        hux() %>%
                        merge_repeated_rows(col=1) %>%
                        set_rotation(col=1, value=90) %>%
                        set_valign(col=1, value="middle") %>%
                        set_header_rows(1, TRUE) %>%
                        set_contents(1,1, value = "") %>%

                        set_caption(anova_02_full_caption) %>%

                        set_bottom_padding(row=8, value=8) %>%
                        set_top_padding(row=9, value=8) %>%

                        set_bottom_padding(row=8, value=15) %>%
                        set_top_padding(row=9, value=15) %>%
                        theme_article()


write(to_latex(anova_02_full_table), paste(tables_path, "anova_02_full.tex", sep="/"))



############

anova_03_full_caption  = "Results of the 2-way ANOVA (condition x stimulus type) for repeated measures. Only fronto included."


anova_03_full_table_1 = rbind(anova_03_100_table %>% add_column(Electrodes = "Frontal", .before = 1), anova_04_100_table %>% add_column(Electrodes = "Mastoids", .before =1)) 
anova_03_full_table_2 = rbind(anova_03_150_table %>% add_column(Electrodes = "Frontal", .before = 1), anova_04_150_table %>% add_column(Electrodes = "Mastoids", .before =1)) 

anova_03_full_table = rbind(anova_03_full_table_1 %>% add_column(SOA = "100 ms", .before = 1), anova_03_full_table_2 %>% add_column(SOA = "150 ms", .before =1)) %>%  hux() %>%
                        set_number_format(col=7, value=list(fm_pvalue)) %>%
                        merge_repeated_rows(col=c(1,2)) %>%
                        set_rotation(col=c(1,2), value=90) %>%
                        set_valign(col=c(1,2), value="middle") %>%
                        set_header_rows(1, TRUE) %>%
                        set_contents(1,1, value = "") %>%
                        set_contents(1,2, value = "") %>%

                        set_caption(anova_02_full_caption) %>%

                        set_bottom_padding(row=4, value=8) %>%

                        set_top_padding(row=5, value=8) %>%
                        set_bottom_padding(row=7, value=15) %>%

                        set_top_padding(row=8, value=15) %>%
                        set_bottom_padding(row=10, value=8) %>%

                        set_top_padding(row=11, value=8) %>%

                        #set_bottom_padding(row=8, value=15) %>%
                        #set_top_padding(row=9, value=15) %>%
                        theme_article()


write(to_latex(anova_03_full_table), paste(tables_path, "anova_03_full.tex", sep="/"))


xtable(anova_03_full_table)

# Write Vars
yaml::write_yaml(variables, "/home/marc/ba-thesis/input/vars.yaml")




variables

,SOA,Electrodes,Effect,DFn,DFd,F,p,p<.05,ges
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,,Effect,DFn,DFd,F,p,p<.05,ges
1.1,100 ms,Frontal,Condition,1,19,0.16,0.694,,0.003
2,100 ms,Frontal,StimulusType,1,19,0.006,0.938,,1.5e-05
3,100 ms,Frontal,Condition x StimulusType,1,19,16.747,0.000621,*,0.013
4,100 ms,Mastoids,Condition,1,19,1.283,0.272,,0.014
5,100 ms,Mastoids,StimulusType,1,19,1.211,0.285,,0.004
6,100 ms,Mastoids,Condition x StimulusType,1,19,2.365,0.141,,0.009
7,150 ms,Frontal,Condition,1,22,0.947,0.341,,0.006
8,150 ms,Frontal,StimulusType,1,22,22.674,9.4e-05,*,0.038


$anova_00_soa
[1] "$F(1,41) = 0.19$, $p = 0.6680$, $\\eta_{gen}=0.0030$"

$anova_02_100_condition
[1] "$F(1,19) = 0.83$, $p = 0.3730$"

$anova_02_100_stimulustype
[1] "$F(1,19) = 1.05$, $p = 0.3180$"

$anova_02_100_electrode
[1] "$F(1,19) = 0.04$, $p = 0.8520$"

$anova_02_100_condition_stimulustype
[1] "$F(1,19) = 0.05$, $p = 0.8230$"

$anova_02_100_condition_electrode
[1] "$F(1,19) = 0.76$, $p = 0.3930$"

$anova_02_100_stimulustype_electrode
[1] "$F(1,19) = 0.80$, $p = 0.3830$"

$anova_02_100_condition_stimulustype_electrode
[1] "$F(1,19) = 7.53$, $p = 0.0130$"

$anova_01_100_condition
[1] "$F(1,19) = 0.16$, $p = 0.6940$"

$anova_01_100_stimulustype
[1] "$F(1,19) = 0.01$, $p = 0.9380$"

$anova_01_100_condition_stimulustype
[1] "$F(1,19) = 16.75$, $p = 0.0006$"

$anova_02_150_condition
[1] "$F(1,22) = 0.08$, $p = 0.7800$"

$anova_02_150_stimulustype
[1] "$F(1,22) = 0.32$, $p = 0.5790$"

$anova_02_150_electrode
[1] "$F(1,22) = 0.04$, $p = 0.8540$"

$anova_02_150_condition_stimulustype
[1] "$F(1,22) = 0.16$, $p = 0.6930$"

$anova_02_150_condition_electrode
[1] "$F(1,22) = 1.13$, $p = 0.2990$"

$anova_02_150_stimulustype_electrode
[1] "$F(1,22) = 20.76$, $p = 0.0002$"

$anova_02_150_condition_stimulustype_electrode
[1] "$F(1,22) = 0.05$, $p = 0.8190$"

$anova_01_150_condition
[1] "$F(1,22) = 0.95$, $p = 0.3410$"

$anova_01_150_stimulustype
[1] "$F(1,22) = 22.67$, $p = 0.0001$"

$anova_01_150_condition_stimulustype
[1] "$F(1,22) = 0.03$, $p = 0.8680$"

In [13]:
## ANOVA Plot
library(ggthemes)

plot = ggplot(anova_02_100_data, aes(y=MeanAmplitude, x=StimulusType, color=Electrode)) +
    facet_wrap(~ Condition) +
    stat_summary(fun.data = mean_se, geom = "errorbar") +
    stat_summary(aes(group=Electrode), fun.y=mean, geom="line") +
    geom_rangeframe(color="black") +
    theme_tufte() +
    theme(text=element_text(size=8,  family="Sans")) 

ggsave("/home/marc/ba-thesis/input/figures/fig9.pdf", plot, width = 6.25, height = 2.5, units = "in")
ggsave("/home/marc/ba-thesis/input/figures/fig9.png", plot, width = 6.25, height = 2.5, units = "in")


Warning message:
“`fun.y` is deprecated. Use `fun` instead.”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message

ERROR: Error in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, : invalid font type


In [131]:
install.packages("ggthemes")

Installing package into ‘/home/marc/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)



In [26]:
sta_data_150 = n1_mean_amplitudes_df %>% filter(SOA == "150" & Electrode == "fronto_pooled" & StimulusType != "B-5" & StimulusType != "A-5") %>% 
            mutate(StimulusType = as.integer( substr(StimulusType, 3, 3) )) %>% droplevels()

model1 = lmer(MeanAmplitude ~ Condition +  (1 | Participant), data=sta_data_150, REML=FALSE) 
model2 = lmer(MeanAmplitude ~ Condition + StimulusType + Condition * StimulusType + (1 | Participant), data = sta_data_150, REML=FALSE)


model3 = lmer(MeanAmplitude ~ StimulusType +  (1 | Participant), data=sta_data_150, REML=FALSE) 
model4 = lmer(MeanAmplitude ~ Condition + StimulusType + Condition * StimulusType + (1 | Participant), data = sta_data_150, REML=FALSE)


anova(model1, model2)

anova(model2)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model1,4,-4608.037,-4595.177,2308.018,-4616.037,NA,NA,NA
model2,6,-4644.729,-4625.440,2328.365,-4656.729,40.69283,2,1.457685e-09


,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
Condition,1.902755e-12,1.902755e-12,1,1.497971,4.8247220,0.20019797
StimulusType,1.794079e-11,1.794079e-11,1,1.497971,45.4915787,0.04243801
Condition:StimulusType,3.178294e-13,3.178294e-13,1,1.497971,0.8059044,0.49000622


In [36]:
xtable(anova_04_150_table)

,Effect,DFn,DFd,F,p,p<.05,ges
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,Condition,1,22,0.206,0.655,,0.00100
2,StimulusType,1,22,6.561,0.018,*,0.01800
3,Condition x StimulusType,1,22,0.122,0.730,,0.00028
